## НАШ ПРОЕКТИК

# 1. Получение данных


In [2]:
import requests
import pandas as pd
from datetime import datetime

# Геокоординаты для нужных городов
city_coords = {
    'Moscow': (55.7558, 37.6176),
    'Saint_Petersburg': (59.9343, 30.3351),
    'Nizhny_Novgorod': (56.2965, 43.9361),
    'Kazan': (55.7963, 49.1088),
    'Ufa': (54.7388, 55.9721),
    'Novosibirsk': (55.0084, 82.9357),
    'Vladivostok': (43.1198, 131.8869)
}

# Функция получения данных из Open-Meteo
def get_weather_open_meteo(city: str, lat: float, lon: float,
                           start_date='2023-01-01', end_date='2023-12-31',
                           hourly=True) -> pd.DataFrame:
    base_url = 'https://archive-api.open-meteo.com/v1/archive'
    params = {
        'latitude': lat,
        'longitude': lon,
        'start_date': start_date,
        'end_date': end_date,
        'timezone': 'auto'
    }

    if hourly:
        params['hourly'] = 'temperature_2m'
    else:
        params['daily'] = 'temperature_2m_mean'

    response = requests.get(base_url, params=params)
    if response.status_code != 200:
        raise Exception(f'Ошибка {response.status_code}: {response.text}')
    
    data = response.json()
    
    if hourly:
        time_list = data['hourly']['time']
        temp_list = data['hourly']['temperature_2m']
    else:
        time_list = data['daily']['time']
        temp_list = data['daily']['temperature_2m_mean']
    
    df = pd.DataFrame({
        'datetime': pd.to_datetime(time_list),
        'temperature': temp_list
    })
    df['city'] = city
    return df

# Сбор всех данных
hourly_data = []
daily_data = []

for city, (lat, lon) in city_coords.items():
    print(f"[⏱] Загружаем почасовые данные для: {city}")
    df_hourly = get_weather_open_meteo(city, lat, lon, hourly=True)
    hourly_data.append(df_hourly)
    
    print(f"[📆] Загружаем ежедневные данные для: {city}")
    df_daily = get_weather_open_meteo(city, lat, lon, hourly=False)
    daily_data.append(df_daily)

# Объединяем все города
df_hourly_all = pd.concat(hourly_data).reset_index(drop=True)
df_daily_all = pd.concat(daily_data).reset_index(drop=True)

# Агрегация по месяцам
df_monthly_all = (
    df_daily_all.copy()
    .assign(month=lambda df: df['datetime'].dt.to_period('M'))
    .groupby(['city', 'month'])
    .agg(temperature=('temperature', 'mean'))
    .reset_index()
)
df_monthly_all['month'] = df_monthly_all['month'].astype(str)

# Сохраняем все 3 таблицы
df_hourly_all.to_csv("weather_2023_hourly.csv", index=False)
df_daily_all.to_csv("weather_2023_daily.csv", index=False)
df_monthly_all.to_csv("weather_2023_monthly.csv", index=False)

# Функция для присвоения сезона по месяцу
def get_season(month: int) -> str:
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'autumn'

# Почасовые данные
df_hourly_all['month'] = df_hourly_all['datetime'].dt.month
df_hourly_all['season'] = df_hourly_all['month'].apply(get_season)

# Ежедневные данные
df_daily_all['month'] = df_daily_all['datetime'].dt.month
df_daily_all['season'] = df_daily_all['month'].apply(get_season)

# Ежемесячные данные
df_monthly_all['month_num'] = pd.to_datetime(df_monthly_all['month']).dt.month
df_monthly_all['season'] = df_monthly_all['month_num'].apply(get_season)

# # Проверка
# print("\n📌 Почасовые данные с сезонами:")
# print(df_hourly_all[['datetime', 'temperature', 'city', 'season']].head())

# print("\n📌 Ежедневные данные с сезонами:")
# print(df_daily_all[['datetime', 'temperature', 'city', 'season']].head())

# print("\n📌 Ежемесячные данные с сезонами:")
# print(df_monthly_all[['month', 'temperature', 'city', 'season']].head())

[⏱] Загружаем почасовые данные для: Moscow
[📆] Загружаем ежедневные данные для: Moscow
[⏱] Загружаем почасовые данные для: Saint_Petersburg
[📆] Загружаем ежедневные данные для: Saint_Petersburg
[⏱] Загружаем почасовые данные для: Nizhny_Novgorod
[📆] Загружаем ежедневные данные для: Nizhny_Novgorod
[⏱] Загружаем почасовые данные для: Kazan
[📆] Загружаем ежедневные данные для: Kazan
[⏱] Загружаем почасовые данные для: Ufa
[📆] Загружаем ежедневные данные для: Ufa
[⏱] Загружаем почасовые данные для: Novosibirsk
[📆] Загружаем ежедневные данные для: Novosibirsk
[⏱] Загружаем почасовые данные для: Vladivostok
[📆] Загружаем ежедневные данные для: Vladivostok


# 2. Построение сетей случайных величин 

In [3]:
import networkx as nx

def build_correlation_graphs(df: pd.DataFrame, time_col: str, value_col: str = 'temperature') -> dict:
    results = {}
    for season in ['winter', 'spring', 'summer', 'autumn']:
        df_season = df[df['season'] == season]

        # pivot: time x city
        pivot_df = df_season.pivot(index=time_col, columns='city', values=value_col)

        # drop rows with missing values
        pivot_df = pivot_df.dropna()

        for method in ['pearson', 'kendall']:
            corr_matrix = pivot_df.corr(method=method)

            # Построим граф
            G = nx.Graph()
            for city in corr_matrix.columns:
                G.add_node(city)

            for i in corr_matrix.columns:
                for j in corr_matrix.columns:
                    if i != j:
                        weight = corr_matrix.loc[i, j]
                        G.add_edge(i, j, weight=weight)

            results[(season, method)] = G

    return results

In [6]:
# Пример: построить графы для почасовых данных
hourly_graphs = build_correlation_graphs(
    df=df_hourly_all,
    time_col='datetime'
)

# То же для дневных и месячных
daily_graphs = build_correlation_graphs(df_daily_all, time_col='datetime')
monthly_graphs = build_correlation_graphs(df_monthly_all.rename(columns={'month': 'datetime'}), time_col='datetime')

In [ ]:
import matplotlib.pyplot as plt

def draw_graph_for_season(graphs_dict, season='spring', method='pearson', threshold=None):
    G = graphs_dict.get((season, method))
    if not G or len(G.nodes) == 0:
        print(f"⚠️ Нет данных для {season.title()} ({method})")
        return

    pos = nx.spring_layout(G, seed=42)

    # Применим фильтр по порогу корреляции
    edges = [(u, v) for u, v, d in G.edges(data=True)
             if threshold is None or abs(d['weight']) >= threshold]
    if not edges:
        print(f"⚠️ Нет рёбер выше порога {threshold} для {season.title()}")
        return

    weights = [G[u][v]['weight'] for u, v in edges]
    abs_weights = [abs(w) for w in weights]

    # Цвета рёбер: от красного (-1) до зелёного (+1)
    edge_colors = [plt.cm.RdYlGn((w + 1) / 2) for w in weights]
    edge_labels = {(u, v): f"{G[u][v]['weight']:.2f}" for u, v in edges}

    plt.figure(figsize=(8, 6))
    nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=1500)
    nx.draw_networkx_labels(G, pos, font_size=11)
    nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color=edge_colors,
                           width=[2 + 4 * w for w in abs_weights])
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=9)

    plt.title(f"{season.title()} — {method.title()} Correlation", fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    # Пример: сравнение по сезонам для дневных наблюдений
draw_graph_for_season(daily_graphs, method='pearson')
draw_graph_for_season(daily_graphs, method='kendall')

# 3. Построение графов с порогом корреляции + клики и независимые множества

In [14]:
#функция отсечения графа 

from networkx.algorithms.clique import find_cliques
from networkx.algorithms.approximation import maximum_independent_set

def threshold_graph(G: nx.Graph, threshold: float) -> nx.Graph:
    H = nx.Graph()
    H.add_nodes_from(G.nodes)
    for u, v, data in G.edges(data=True):
        if abs(data['weight']) >= threshold:
            H.add_edge(u, v, weight=data['weight'])
    return H

# функция которая находит клики + независимые множества
def analyze_graph(G: nx.Graph, threshold: float):
    G_thresh = threshold_graph(G, threshold)

    cliques = list(find_cliques(G_thresh))  # Все клики (не обязательно максимальные)
    max_clique_size = max(len(c) for c in cliques) if cliques else 0
    largest_cliques = [c for c in cliques if len(c) == max_clique_size]

    independent_set = maximum_independent_set(G_thresh)

    return {
        'thresholded_graph': G_thresh,
        'cliques': cliques,
        'largest_cliques': largest_cliques,
        'independent_set': independent_set
    }

In [15]:
# это пример использования функции выделения клик + независимых множеств

thresholds = [0.2, 0.4, 0.7]

for season in ['winter', 'spring', 'summer', 'autumn']:
    for method in ['pearson', 'kendall']:
        G = hourly_graphs.get((season, method))
        if G is None:
            continue
        print(f"\n--- {season.title()} | {method.title()} ---")
        for t in thresholds:
            result = analyze_graph(G, t)
            print(f"\nПорог: {t}")
            print(f"🔹 Кол-во вершин в кликах: {len(result['cliques'])}")
            print(f"🔹 Наибольшие клики: {result['largest_cliques']}")
            print(f"🔹 Независимое множество: {result['independent_set']}")


--- Winter | Pearson ---

Порог: 0.2
🔹 Кол-во вершин в кликах: 3
🔹 Наибольшие клики: [['Saint_Petersburg', 'Moscow', 'Nizhny_Novgorod', 'Kazan', 'Ufa']]
🔹 Независимое множество: {'Moscow', 'Novosibirsk'}

Порог: 0.4
🔹 Кол-во вершин в кликах: 4
🔹 Наибольшие клики: [['Kazan', 'Nizhny_Novgorod', 'Moscow', 'Saint_Petersburg'], ['Kazan', 'Nizhny_Novgorod', 'Moscow', 'Ufa']]
🔹 Независимое множество: {'Saint_Petersburg', 'Ufa'}

Порог: 0.7
🔹 Кол-во вершин в кликах: 5
🔹 Наибольшие клики: [['Nizhny_Novgorod', 'Kazan', 'Moscow'], ['Nizhny_Novgorod', 'Kazan', 'Ufa']]
🔹 Независимое множество: {'Saint_Petersburg', 'Vladivostok', 'Kazan', 'Novosibirsk'}

--- Winter | Kendall ---

Порог: 0.2
🔹 Кол-во вершин в кликах: 4
🔹 Наибольшие клики: [['Kazan', 'Nizhny_Novgorod', 'Moscow', 'Saint_Petersburg'], ['Kazan', 'Nizhny_Novgorod', 'Moscow', 'Ufa']]
🔹 Независимое множество: {'Saint_Petersburg', 'Ufa'}

Порог: 0.4
🔹 Кол-во вершин в кликах: 5
🔹 Наибольшие клики: [['Nizhny_Novgorod', 'Kazan', 'Moscow'], ['N

In [ ]:
#функция визуализации отсеченного графа
def draw_thresholded_graph(G: nx.Graph, threshold=0.7, title=''):
    G_thresh = threshold_graph(G, threshold)
    
    if G_thresh.number_of_edges() == 0:
        print(f"⚠️ Нет рёбер выше порога {threshold}")
        return

    pos = nx.spring_layout(G_thresh, seed=42)

    edges = list(G_thresh.edges(data=True))
    weights = [d['weight'] for _, _, d in edges]
    abs_weights = [abs(w) for w in weights]
    edge_colors = [plt.cm.RdYlGn((w + 1) / 2) for w in weights]
    edge_labels = {(u, v): f"{w:.2f}" for u, v, d in edges if (w := d['weight'])}

    plt.figure(figsize=(8, 6))
    nx.draw_networkx_nodes(G_thresh, pos, node_color='lightblue', node_size=1500)
    nx.draw_networkx_labels(G_thresh, pos, font_size=11)
    nx.draw_networkx_edges(G_thresh, pos, edgelist=[(u, v) for u, v, _ in edges],
                           edge_color=edge_colors,
                           width=[2 + 4 * w for w in abs_weights])
    nx.draw_networkx_edge_labels(G_thresh, pos, edge_labels=edge_labels, font_size=9)
    
    plt.title(title or f"Thresholded Graph (≥ {threshold})", fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
#визуализация зимнего графа пирсона из почасовых данных при пороге 0.7:
G = hourly_graphs[('winter', 'pearson')]
draw_thresholded_graph(G, threshold=0.7, title='Winter (Pearson), Threshold ≥ 0.7')

# 4. Анализ гипотезы

# 5. Выводы